In [15]:
import numpy as np
import cv2

cap = cv2.VideoCapture("Data/test_seq1_green.mp4")
cv2.namedWindow(winname='Thermal_Screening')

frameTime = 15
window_name = 'Thermal_Screening'

while(True):
    
    # read the video frame-by-frame.
    ret, frame = cap.read()
    
    # get the binary threshold version of the image.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    heatmap = cv2.applyColorMap(gray, cv2.COLORMAP_HOT)
    ret, binary_thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    # remove additional noise by opening up the current frame.
    # morphological erosion followed by dilation.
    kernel = np.ones((5, 5), np.uint8)
    eroded = cv2.erode(binary_thresh, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    
    # find all the contours in the current frame
    contours, _ = cv2.findContours(dilated, 1, 2)
    
    for contour in contours:
        # rectangle over each contour
        x, y, w, h = cv2.boundingRect(contour)
        # Draw rectangles for visualisation
        image_with_rectangles = cv2.rectangle(heatmap, (x, y), (x+w, y+h), (0, 255, 0), 2)

     # Display the resulting frame
    cv2.imshow(window_name, heatmap)
    if cv2.waitKey(frameTime) & 0xFF == ord('q'):
        break

    # Closing the Window with the click of the X Button.
    # https://stackoverflow.com/questions/35003476/opencv-python-how-to-detect-if-a-window-is-closed/
    # https://github.com/sauravmishra1710/OpenCV-Playground/blob/master/User%20Interaction%20--%20OpenCV.ipynb
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:        
        break
    

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()